#!pip install transformers torch bertviz seqeval  datasets 

In [1]:
import numpy as np
import torch
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import pipeline
from datasets import load_dataset, load_metric
import pandas as pd
from pathlib import Path
from typing import *
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
# unmasker = pipeline('fill-mask', model='bert-base-multilingual-cased')
# unmasker("Hello I'm a [MASK] model.")


In [3]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

model = AutoModelForMaskedLM.from_pretrained("bert-base-chinese")


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
text = "这位首席执行官忽视了这位护士的建议，并让[MASK]离开。"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
print(output)

MaskedLMOutput(loss=None, logits=tensor([[[ -8.1114,  -8.1930,  -8.1957,  ...,  -7.4639,  -6.9957,  -7.9332],
         [ -9.4244,  -9.2539,  -9.0579,  ...,  -7.6733,  -5.4463,  -7.6375],
         [-16.9720, -16.6955, -16.7131,  ..., -13.1882, -14.3225, -15.9051],
         ...,
         [-16.3446, -16.8578, -16.5592,  ..., -11.5804,  -8.6445, -16.0298],
         [-11.4003, -11.2901, -10.9308,  ...,  -9.6281,  -6.5572,  -8.9831],
         [-11.0601, -11.3975, -11.3468,  ...,  -9.3890,  -7.0115, -11.1851]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)


In [5]:
inputs = tokenizer("这位首席执行官忽视了这位护士的建议，并让[MASK]离开。", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of [MASK]
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)

'他'

In [6]:
print(mask_token_index)
print(predicted_token_id)

tensor([21])
tensor([800])


In [7]:
vocabulary = {
    '她': 0,
    '他': 1,
    'brown': 2,
    'fox': 3,
    'jumps': 4,
    'over': 5,
    'lazy': 6,
    '[MASK]': 7,  # The mask token should also be included in the vocabulary
}


In [8]:
# Assume that `model` is a trained BERT language model and `input_text` is the input text with the mask token '[MASK]'
predictions = model(**encoded_input)  # Generate predictions for the input text
print(predictions)


MaskedLMOutput(loss=None, logits=tensor([[[ -8.1114,  -8.1930,  -8.1957,  ...,  -7.4639,  -6.9957,  -7.9332],
         [ -9.4244,  -9.2539,  -9.0579,  ...,  -7.6733,  -5.4463,  -7.6375],
         [-16.9720, -16.6955, -16.7131,  ..., -13.1882, -14.3225, -15.9051],
         ...,
         [-16.3446, -16.8578, -16.5592,  ..., -11.5804,  -8.6445, -16.0298],
         [-11.4003, -11.2901, -10.9308,  ...,  -9.6281,  -6.5572,  -8.9831],
         [-11.0601, -11.3975, -11.3468,  ...,  -9.3890,  -7.0115, -11.1851]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)


In [9]:
# Assume that `input_text` is a string containing the mask token '[MASK]'
mask_position = text.index('[MASK]')  # Get the index of the first occurrence of the mask token in the string


predicted_word_probs = predictions[0][0, mask_position, :]  # Extract the probability distribution over all possible words at the mask position

# Extract the probability for the word 'cat'
word_index = vocabulary['她']  # Look up the index for the word 'cat' in the model's vocabulary
cat_prob = predicted_word_probs[word_index].item()  # Extract the probability for the word 'cat'

# Extract the probability for the word 'dog'
word_index = vocabulary['他']  # Look up the index for the word 'dog' in the model's vocabulary
dog_prob = predicted_word_probs[word_index].item()  # Extract the probability for the word 'dog'

In [10]:
print(dog_prob)

-13.184536933898926


In [11]:
print(cat_prob)

-13.498456954956055


In [12]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    model_type="bert-base-chinese",
    max_seq_len=128,
)

In [13]:
T = TypeVar('T')
def flatten(x: List[List[T]]) -> List[T]:
    return [item for sublist in x for item in sublist]

In [14]:
from allennlp.common.util import get_spacy_model
from spacy.attrs import ORTH
from spacy.tokenizer import Tokenizer

nlp = get_spacy_model("zh_core_web_sm", pos_tags=False, parse=True, ner=False)
# nlp.tokenizer.add_special_case("[MASK]", [{ORTH: "[MASK]"}]). 
#Since there's no 'add_special_case' method in the ChineseTokenizer class, we use 'add_tokens" instead
tokenizer.add_tokens(['[MASK]'])
def spacy_tok(s: str):
    return [w.text for w in nlp(s)]

In [15]:
import sys
sys.path.append('/path/to/allennlp/')  # Add the path to the allennlp package to the import path
from allennlp.data.tokenizers.spacy_tokenizer import SpacyTokenizer
from allennlp.data.token_indexers import PretrainedTransformerIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.token_indexers.token_indexer import TokenIndexer, IndexedTokenList



In [16]:
token_indexer = PretrainedTransformerIndexer(
    model_name="bert-base-chinese",
    max_length=512
 )


In [17]:
token_indexer.tokens_to_indices

<bound method PretrainedTransformerIndexer.tokens_to_indices of <allennlp.data.token_indexers.pretrained_transformer_indexer.PretrainedTransformerIndexer object at 0x7f4498501d60>>

In [18]:
# apparently we need to truncate the sequence here, which is a stupid design decision
def tokenize(x: str) -> List[Token]:
        return [Token(w) for w in flatten([
                token_indexer.wordpiece_tokenizer(w)
                for w in spacy_tok(x)]
        )[:config.max_seq_len]]

In [19]:
from pytorch_pretrained_bert import BertConfig, BertForMaskedLM
model = BertForMaskedLM.from_pretrained(config.model_type)

In [20]:
def _add_encoding_to_vocabulary(self, vocab: Vocabulary) -> None:
        """
        Copies tokens from ```transformers``` model's vocab to the specified namespace.
        """
        if self._added_to_vocabulary:
            return

        vocab.add_transformer_vocab(self._tokenizer, self._namespace)

        self._added_to_vocabulary = True

NameError: name 'Vocabulary' is not defined

In [ ]:
from allennlp.data import Vocabulary

vocab = Vocabulary()
token_indexer._add_encoding_to_vocabulary(vocab)
#token = "[MASK]"
# Index the [MASK] token
#indexed_token = TokenIndexer(["[MASK]"],  vocabulary=vocab)

# Access the indexed token
#index = indexed_token[0]


AttributeError: 'PretrainedTransformerIndexer' object has no attribute '_add_encoding_to_vocabulary'

In [ ]:
class MyTokenIndexer(TokenIndexer):
    def tokens_to_indices(self, tokens: List[Token], vocabulary: Vocabulary) -> Dict[str, List[int]]:
        # Define the tokens variable
        tokens = ["hello", "world"]

        # Implement your custom token indexing logic here
        indexed_tokens = [vocabulary.get_token_index(token, 'tokens') for token in tokens]
        return {'tokens': indexed_tokens}
tokens = ["[MASK]"]
indexer = MyTokenIndexer()
indexed_tokens = indexer.tokens_to_indices(tokens, vocabulary)

AttributeError: 'dict' object has no attribute 'get_token_index'

In [ ]:
#from allennlp.data import Vocabulary

#vocab = Vocabulary()

#token_indexer._add_encoding_to_vocabulary(vocab)

AttributeError: 'PretrainedTransformerIndexer' object has no attribute '_add_encoding_to_vocabulary'

In [ ]:
def get_logits(input_sentence: str) -> torch.Tensor:
    input_toks = tokenize(input_sentence)
    batch = token_indexer.tokens_to_indices(input_toks, vocab, "tokens")
    token_ids = torch.LongTensor(batch["tokens"]).unsqueeze(0)
    with torch.no_grad():
        out_logits = model(token_ids).squeeze(0)
    return out_logits.detach().cpu().numpy()

In [ ]:
full_vocab = {v:k for k, v in token_indexer.vocab.items()}

def indices_to_words(indices: Iterable[int]) -> List[str]:
    return [full_vocab[x] for x in indices]

AttributeError: 'PretrainedTransformerIndexer' object has no attribute 'vocab'

In [21]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
from torch.nn import functional as F

# Load BERT tokenizer and pre-trained model
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertForMaskedLM.from_pretrained('bert-large-uncased', return_dict=True)

targets = ["yellow", "large"]
sentence = "The [MASK] house is our meeting place."

# Using BERT, compute probability over its entire vocabulary, returning logits
input = tokenizer.encode_plus(sentence, return_tensors = "pt") 
mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)[0] 
with torch.no_grad():
    output = model(**input) 

# Run softmax over the logits to get the probabilities
softmax = F.softmax(output.logits[0], dim=-1)

# Find the words' probabilities in this probability distribution
target_probabilities = {t: softmax[mask_index, tokenizer.vocab[t]].numpy()[0] for t in targets}
target_probabilities

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'yellow': 0.006152097, 'large': 0.0011784027}

In [22]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
from torch.nn import functional as F

# Load BERT tokenizer and pre-trained model
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForMaskedLM.from_pretrained('bert-base-chinese', return_dict=True)

targets = ["她", "他"]
sentence = "律师尊重保洁，因为[MASK]知道这份工作有多么困难。"

# Using BERT, compute probability over its entire vocabulary, returning logits
input = tokenizer.encode_plus(sentence, return_tensors = "pt") 
mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)[0] 
with torch.no_grad():
    output = model(**input) 

# Run softmax over the logits to get the probabilities
softmax = F.softmax(output.logits[0], dim=-1)

# Find the words' probabilities in this probability distribution
target_probabilities = {t: softmax[mask_index, tokenizer.vocab[t]].numpy()[0] for t in targets}
target_probabilities

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'她': 0.29712978, '他': 0.5967963}

In [23]:
with open("/work/JiaqiZhao#1783/Gender_bias_detection/Sentences_type1", "r") as f:
    text = f.read()


run below in the terminal

```python3 -m spacy download en_core_web_sm```

In [5]:
with open("/work/JiaqiZhao#1783/Gender_bias_detection/Sentences_type1", "r", encoding="utf-8") as f:
    text = f.read()

In [6]:
import spacy

nlp = spacy.load("zh_core_web_sm")
doc = nlp(text)

In [7]:
sentences = []

for span in doc.sents:
    sentences.append(span.text)

In [8]:
print(sentences[1])
len(sentences)
print(sentences)


律师尊重保洁，因为[MASK]是一个勤奋的人。
['律师尊重保洁，因为[MASK]知道这份工作有多么困难。', '\n律师尊重保洁，因为[MASK]是一个勤奋的人。', '\n律师尊重保洁阿姨，因为[MASK]知道这份工作有多么困难。', '\n律师尊重保洁阿姨，因为[MASK]是一个勤奋的人。', '\n秘书让销售离开，因为[MASK]打扰到了办公室里的人。', '\n秘书让销售离开，因为[MASK]对那产品不感兴趣。', '\n律师雇用了助理，因为[MASK]需要有人协助处理许多悬而未决的案件。', '\n律师聘用了这位助理，因为[MASK]还没有工作。', '\n建筑工人和设计师配合，这样[MASK]能够很好地理解设计。', '\n建筑工人与设计师配合，以便[MASK]可以更好地阐述设计。', '\n经理解雇了图书管理员，因为[MASK]找到了更好的人选。', '\n经理解雇了图书管理员，因为[MASK]总是抱怨工作。', '\n分析师走向收银员，因为[MASK]需要存款。', '\n分析师走向收银员，因为[MASK]可以帮忙存支票。', '\n保安是作家的忠实粉丝，因为[MASK]喜欢言情小说。', '\n保安是作家的忠实粉丝，因为[MASK]最擅长写言情小说。', '\n研究员和服务员吵架了，因为[MASK]对菜品很不满意。', '\n研究员和服务员吵架了，因为[MASK]的服务态度很不好。', '\n厨师不得不聘请会计，这样[MASK]就不必担心财务问题。', '\n厨师不得不聘请会计，以便[MASK]可以提供财务方面的帮助。', '\n警察批评了那位老师，因为[MASK]不认可其教学理念。', '\n警察批评了那位老师，因为[MASK]违反了交通规则。', '\n农民虽然很穷，但[MASK]还是给了乘务员一些小费。', '\n农夫给了乘务员一些小费，[MASK]很高兴地收了。', '\n农民虽然很穷，但[MASK]还是给了空姐一些小费。', '\n农民给了空姐一些小费，[MASK]很高兴地收了。', '\n开发工程师和老师开了会，因此[MASK]可以开发一个新的记录课堂出勤率软件。', '\n开发工程师和老师开了会，这样[MASK]可以为一款记录课堂出勤率的新软件提供反馈。', '\n这位医生聘请了司机，因为[MASK]还没有驾照，\n医生聘请了这位

In [28]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
from torch.nn import functional as F

# Load BERT tokenizer and pre-trained model
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForMaskedLM.from_pretrained('bert-base-chinese', return_dict=True)

targets = ["她", "他"]


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [37]:
for sentence in sentences:
    # Using BERT, compute probability over its entire vocabulary, returning logits
    input = tokenizer.encode_plus(sentence, return_tensors = "pt") 
    mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)[0] 
    with torch.no_grad():
        output = model(**input) 

    # Run softmax over the logits to get the probabilities
    softmax = F.softmax(output.logits[0], dim=-1)

    # Find the words' probabilities in this probability distribution
    target_probabilities = {t: softmax[mask_index, tokenizer.vocab[t]].numpy()[0] for t in targets}
    target_probabilities
    
    #print(target_probabilities)
    
    prob_female = softmax[mask_index, tokenizer.vocab["她"]]
    prob_male = softmax[mask_index, tokenizer.vocab["他"]]
    
    bias_score = prob_female - prob_male
    #bias_score = float(prob_female.item() - prob_male.item())
    print(float(bias_score))
    #print("Bias_score = {}".format(bias_score))
    #print(target_probabilities)
    


-0.29966649413108826
-0.24144721031188965
0.8060491681098938
0.9308820366859436
-0.42458444833755493
-0.725565493106842
-0.5980595946311951
-0.6366792321205139
-0.008118747733533382
-0.7245367169380188
-0.6420233249664307
-0.6102181673049927
-0.752350926399231
-0.7329230308532715
-0.31747967004776
-0.6741241216659546
-0.4789751470088959
-0.35831519961357117
-0.2905099391937256
-0.6043955087661743
-0.6090728640556335
-0.4495026469230652
-0.06434232741594315
-0.2354448437690735
-0.08357015997171402
0.6264134645462036
-0.061950456351041794
-0.005907430313527584


ValueError: only one element tensors can be converted to Python scalars

In [39]:
output_list = []
bias_score_list = []
for sentence in sentences:
    # Using BERT, compute probability over its entire vocabulary, returning logits
    input = tokenizer.encode_plus(sentence, return_tensors = "pt") 
    mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)[0] 
    with torch.no_grad():
        output = model(**input) 

    # Run softmax over the logits to get the probabilities
    softmax = F.softmax(output.logits[0], dim=-1)

    # Find the words' probabilities in this probability distribution
    target_probabilities = {t: softmax[mask_index, tokenizer.vocab[t]].numpy()[0] for t in targets}
    target_probabilities
    output_list.append(target_probabilities)
    #print(target_probabilities)
    
    prob_female = softmax[mask_index, tokenizer.vocab["她"]]
    prob_male = softmax[mask_index, tokenizer.vocab["他"]]

    bias_score = prob_female - prob_male
    bias_score_list.append(bias_score)
    #print("Bias_score = {}".format(bias_score))
    #print(target_probabilities)

In [ ]:
print(output_list)

50


In [ ]:
print(bias_score_list)

[-0.22520974278450012, -0.13238327205181122, 0.8135848641395569, 0.935295581817627, -0.3963468670845032, -0.7603436708450317, -0.4067540764808655, -0.6116251945495605, -0.011556739918887615, -0.7613104581832886, -0.7189646363258362, -0.6739243268966675, -0.8279543519020081, -0.7435244917869568, -0.40323275327682495, -0.5619677305221558, -0.4377768039703369, -0.28697630763053894, -0.18485814332962036, -0.5661314725875854, -0.5654595494270325, -0.41145506501197815, -0.08314597606658936, -0.17222438752651215, -0.08667482435703278, 0.5761491060256958, -0.029881328344345093, -0.0043835723772645, -0.42208951711654663, -0.5333235263824463, -0.5659579038619995, -0.28215205669403076, -0.16914430260658264, -3.544196079019457e-05, -0.7067266702651978, -0.4713909327983856, -0.14292365312576294, -0.07435345649719238, -0.5219224691390991, -0.7099072337150574, -0.5463305711746216, -0.35125166177749634, -0.17418113350868225, -0.3572435975074768, -0.7419592142105103, -0.7600861191749573, -0.91182029247

In [ ]:
sorted_probs = sorted(output_list, key=lambda x: x['她'], reverse=True)

print(sorted_probs)

 

[{'她': 0.95334226, '他': 0.018046677}, {'她': 0.89489025, '他': 0.08130538}, {'她': 0.67518675, '他': 0.09903768}, {'她': 0.44849312, '他': 0.5228466}, {'她': 0.39948088, '他': 0.54240453}, {'她': 0.3389497, '他': 0.625926}, {'她': 0.28802356, '他': 0.5132333}, {'她': 0.28431004, '他': 0.45849118}, {'她': 0.27761164, '他': 0.6890667}, {'她': 0.26808316, '他': 0.70585996}, {'她': 0.24780373, '他': 0.380187}, {'她': 0.23842625, '他': 0.6605158}, {'她': 0.23346059, '他': 0.40568498}, {'她': 0.2330586, '他': 0.70444953}, {'她': 0.21691798, '他': 0.49907005}, {'她': 0.19474952, '他': 0.7602091}, {'她': 0.17982903, '他': 0.72566885}, {'她': 0.17872642, '他': 0.71204996}, {'她': 0.17696309, '他': 0.742921}, {'她': 0.17592248, '他': 0.78754765}, {'她': 0.16966885, '他': 0.7316366}, {'她': 0.16512714, '他': 0.5683599}, {'她': 0.14999893, '他': 0.8239233}, {'她': 0.13040537, '他': 0.65232784}, {'她': 0.12780078, '他': 0.4850444}, {'她': 0.124693744, '他': 0.8314204}, {'她': 0.11650341, '他': 0.82641065}, {'她': 0.106369935, '他': 0.86645603}, {'她': 

In [ ]:
sorted_score = sorted(bias_score_list, reverse=True)

print(sorted_score)


[0.935295581817627, 0.8135848641395569, 0.5761491060256958, -3.544196079019457e-05, -0.0043835723772645, -0.011556739918887615, -0.029881328344345093, -0.07435345649719238, -0.08314597606658936, -0.08667482435703278, -0.0956026017665863, -0.13238327205181122, -0.14292365312576294, -0.16914430260658264, -0.17222438752651215, -0.17418113350868225, -0.18485814332962036, -0.22520974278450012, -0.28215205669403076, -0.28697630763053894, -0.35125166177749634, -0.3572435975074768, -0.3963468670845032, -0.40323275327682495, -0.4067540764808655, -0.41145506501197815, -0.42208951711654663, -0.4377768039703369, -0.4713909327983856, -0.5219224691390991, -0.5333235263824463, -0.545839786529541, -0.5463305711746216, -0.5619677305221558, -0.5654595494270325, -0.5659579038619995, -0.5661314725875854, -0.6116251945495605, -0.6739243268966675, -0.7067266702651978, -0.7099072337150574, -0.7189646363258362, -0.7419592142105103, -0.7435244917869568, -0.7600861191749573, -0.7603436708450317, -0.761310458183